In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(hise)
quiet_library(dplyr)
quiet_library(purrr)

## Read metadata and labels stored in HISE

In [2]:
meta_uuid <- list("f3db110d-4d9f-45d6-bbf7-02957f23a81e" )

In [3]:
fres <- cacheFiles(meta_uuid)
meta_file <- list.files(paste0("cache/", meta_uuid), full.names = TRUE)

submitting request as query ID first...

retrieving files using fileIDS...



[1] "Initiating file download for filtered_tea_cell_metadata.csv"
[1] "Download successful."


In [4]:
label_uuids <- list(
    "ebd4bee7-2f5d-46e1-b2fc-22157f1b8d04", # CD4 type labels
    "4d6aade9-288c-452f-8f0d-ac59e539f4cc"  # CD8 type labels
)

In [5]:
label_res <- cacheFiles(label_uuids)
label_files <- list.files(
    paste0("cache/",label_uuids),
    recursive = TRUE, full.names = TRUE
)

[1] "Initiating file download for cd4_cell_type_labels_2023-09-05.csv"
[1] "Download successful."
[1] "Initiating file download for cd8_cell_type_labels_2023-09-05.csv"
[1] "Download successful."


In [6]:
meta <- read.csv(meta_file)

In [7]:
labels <- map_dfr(label_files, read.csv)

Join tables and rename for display

In [8]:
all <- left_join(meta, labels)

Joining with `by = join_by(barcodes, treatment, timepoint)`


In [9]:
out_df <- all %>%
  mutate(epi_gate_class = ifelse(
      grepl("cd4", aifi_cell_type), 
      "CD4", "CD8")) %>%
  rename(txn_n_reads = n_reads,
         txn_n_umis = n_umis,
         txn_n_genes = n_genes,
         txn_n_mito_umis = n_mito_umis,
         txn_frac_mito_umis = frac_mito_umis,
         epi_n_adt_umis = adt_umis,
         acc_n_unique = n_unique,
         acc_n_fragments = n_fragments,
         acc_n_mito_frags = n_mito,
         acc_frac_mito_frags = frac_mito) %>%
  select(treatment, timepoint, epi_gate_class, aifi_cell_type,
         barcodes, original_barcodes, well_id,
         txn_n_genes, txn_n_umis, txn_n_reads,
         txn_n_mito_umis, txn_frac_mito_umis,
         epi_n_adt_umis,
         acc_n_unique, acc_n_fragments,
         acc_n_mito_frags, acc_frac_mito_frags)

Save output

In [10]:
out_dir <- "output"
if(!dir.exists("output")) { dir.create("output") }

In [13]:
out_file <- paste0("Supp-Table-02_cell_metadata_", Sys.Date(), ".csv")
write.csv(
    out_df,
    file.path(out_dir, out_file),
    row.names = FALSE,
    quote = FALSE
)

In [14]:
system(paste('gzip', file.path(out_dir, out_file)))

In [15]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] purrr_1.0.2 dplyr_1.1.3 hise_2.16.0

loaded via a namespace (and not attached):
 [1] crayon_1.5.2     vctrs_0.6.3      httr_1.4.7       cli_3.6.1       
 [5] rlang_1.1.1      generics_0.1.3   assertthat_0.2.1 jsonlite_1.8.7  
 [9] glue_1.6.2       RCurl_1.98-1.12  htmltools_0.5.6  IRdisplay_1.1   
[13] IRker